# Example multiprocess data processing  
Thai word tokenize (wongnai corpus)

---
**By Pranpaveen Lay.**

## Download data

In [1]:
!wget https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip

--2022-04-04 15:08:35--  https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip [following]
--2022-04-04 15:08:35--  https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14338706 (14M) [application/zip]
Saving to: ‘review_dataset.zip’

review_dataset.zip  100%[===================>]  13.67M  25.7MB/s    in 0.5s    

2022-04-04 15:08:37 (25.7 MB/s) - ‘review_dataset.zip’ saved [143387

In [3]:
!unzip -q review_dataset.zip -d data

## Read data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/w_review_train.csv',header=None,names=['text'])
df.head(10)

,text
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...


In [3]:
df

,text
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...
...,...
39995,รู้จักร้านนี้จากวงใน ร้านอยู่ในอาคารพาณิชย์ตรง...
39996,ร้านซูชิอาหารญี่ปุ่น อยู่ตรงสะพานลอย เกษตรประต...
39997,"""Cantina Wine Bar & Italian Kitchen"" ร้านเล็กๆ..."
39998,ร้านเค้กน่ารักๆ ตรงชั้นล่างของห้างเซ็นทรัลลาดพ...


## Tokenize with pandas map

In [4]:
from pythainlp.tokenize import word_tokenize

In [5]:
%%time
tok1 = df['text'].map(lambda x : word_tokenize(x, engine="newmm"))

CPU times: user 32.9 s, sys: 207 ms, total: 33.1 s
Wall time: 33.1 s


In [6]:
tok1[:10]

0    [ร้านอาหาร, ใหญ่, มาก, กก, กก, กก,  , \n, เลี้...
1    [อาหาร, ที่นี่, เป็น, อาหารจีน, แคะ, ที่, หากิ...
2    [ปอเปี๊ยะ, สด,  , ทุกวันนี้, รู้สึก, ว่า, หากิ...
3    [รัาน, คัพ, เค้ก, ใน, เมือง, ไทย, มี, ไม่, มาก...
4    [อร่อย, !!!,  , เดินผ่าน, Digital,  , gateway,...
5    [ร้าน, ข้าวต้ม, กระดูก, หมู,  , ปากซอย, พัฒนาก...
6    [วันนี้, ได้, มีโอกาส, ไป, นั่ง, ซด, กาแฟ, ที่...
7    [สารภาพ, ว่า, ไม่, เคย, คิด, จะ, ไป, ต่อ, คิว,...
8    [เมื่อวันก่อน, ไป, เดินเล่น,  , แบบ, ชิวๆ, ๆ, ...
9    [วันก่อน, ไป, เดิน, สยาม,  , หลังจาก, ห่างหาย,...
Name: text, dtype: object

## Tokenize with multiprocess

In [7]:
import multiprocessing as mp
from functools import partial
import time

In [8]:
n_process = 8
pool = mp.Pool(processes=n_process)

In [9]:
%%time
tok2 = pool.map( partial(word_tokenize, engine="newmm") , df['text'].values)

CPU times: user 1.26 s, sys: 392 ms, total: 1.65 s
Wall time: 7.69 s


In [10]:
pd.Series(tok2)[:10]

0    [ร้านอาหาร, ใหญ่, มาก, กก, กก, กก,  , \n, เลี้...
1    [อาหาร, ที่นี่, เป็น, อาหารจีน, แคะ, ที่, หากิ...
2    [ปอเปี๊ยะ, สด,  , ทุกวันนี้, รู้สึก, ว่า, หากิ...
3    [รัาน, คัพ, เค้ก, ใน, เมือง, ไทย, มี, ไม่, มาก...
4    [อร่อย, !!!,  , เดินผ่าน, Digital,  , gateway,...
5    [ร้าน, ข้าวต้ม, กระดูก, หมู,  , ปากซอย, พัฒนาก...
6    [วันนี้, ได้, มีโอกาส, ไป, นั่ง, ซด, กาแฟ, ที่...
7    [สารภาพ, ว่า, ไม่, เคย, คิด, จะ, ไป, ต่อ, คิว,...
8    [เมื่อวันก่อน, ไป, เดินเล่น,  , แบบ, ชิวๆ, ๆ, ...
9    [วันก่อน, ไป, เดิน, สยาม,  , หลังจาก, ห่างหาย,...
dtype: object

## Pandas parallel tools (not work ?)

In [18]:
# import swifter

In [19]:
# %%time
# tok3 = df.swifter.apply(lambda x : word_tokenize(x['text'], engine="newmm"),1,)

In [20]:
# from pandarallel import pandarallel

In [21]:
# pandarallel.initialize()

In [22]:
# %%time
# tok3 = df.parallel_apply(lambda x : word_tokenize(x['text'], engine="newmm"),1)